# Model runs Notebook


In this notebook, various models are trained. Their predictions are then stored for evaluation. In order to obtain robust results, each version of the dataset (original samples only and then its copy combined with augmentation techniques) is split into 10 folds, allowing 10 unique combinations of train/test data to be used.

In total, we have 11 versions of the dataset, with each being split into 10. We end up with at least 110 train+test runs for each model architecture, which creates a lot of computation demand. These tests were run in Google Colab. Results were then stored in a pickled dataframe. Prediction results were then analyzed in [Evaluation Notebook](Evaluation.ipynb).

In [ ]:
!pip install -U scikit-learn pandas
# Original sklearn on Colab is 1.2.2 while 1.3.0 is used on my local machine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 42.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 require

### Step 1: Preparing results dataframe


In [ ]:
#This cell is all about setting results df, and major imports
from google.colab import drive
import pandas as pd
import numpy as np
path = '/content/gdrive/MyDrive/Colab Notebooks/data/'
drive.mount('/content/gdrive')

results = pd.read_pickle(
    path + 'spectrum.pkl').reset_index()[['slice_file_name', 'fold', 'class']]
augment_results = results.copy()
results['source'] = 'o'
augment_results['source'] = 'a'
results = pd.concat([results, augment_results], ignore_index=True, axis=0)
results.to_pickle(path + "results.pkl")

Mounted at /content/gdrive


### Main Execution Part


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from google.colab import drive
path = '/content/gdrive/MyDrive/Colab Notebooks/data/'
drive.mount('/content/gdrive')
names = ['dist', 'mixup', 'imixup', 'room', 'warp', 'delay', 'spectrum']
classes = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark',
           'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']
num_classes = len(classes)

# This parameter is specifying what kind of model is going to be used for the experiment run.
backbone = "RsNet50"


def build_df(augment_name):
    """
    This function is effectively building a working dataframe from unaugmented data and selected (augment_name) augmented dataframe.
    One-hot encoding is also done here.
    """
    original_df = pd.read_pickle(path + 'spectrum.pkl')
    aug_df = pd.read_pickle(path + augment_name + '.pkl')
    enc = OneHotEncoder(handle_unknown='ignore',
                        feature_name_combiner=lambda x, y: y)
    categories_df = pd.DataFrame(enc.fit_transform(original_df['class'].to_numpy(
    ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())
    categories_df = pd.concat(
        [original_df[['slice_file_name', 'fold', 'r']], categories_df], axis=1)
    categories_df['source'] = 'o'
    if augment_name != 'spectrum':
        if augment_name == 'mixup':
            encoding = aug_df.set_index('classID').to_dict()['class']
            aug_df['class2'] = aug_df['classID2'].map(encoding)
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.8
            categories2_df_add = pd.DataFrame(enc.transform(aug_df['class2'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.2
            categories2_df = categories2_df.add(categories2_df_add)
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        elif augment_name == 'imixup':
            encoding = aug_df.set_index('classID').to_dict()['class']
            aug_df['class2'] = aug_df['classID2'].map(encoding)
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.66
            categories2_df_add = pd.DataFrame(enc.transform(aug_df['class2'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())*0.34
            categories2_df = categories2_df.add(categories2_df_add)
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        else:
            categories2_df = pd.DataFrame(enc.transform(aug_df['class'].to_numpy(
            ).reshape(-1, 1)).toarray(), columns=enc.get_feature_names_out())
            categories2_df = pd.concat(
                [aug_df[['slice_file_name', 'fold', 'r']], categories2_df], axis=1)
        categories2_df['source'] = 'a'
        return [categories_df, categories2_df]
    else:
        return [categories_df]


def split(df, fold):
    """
    This function is splitting previously built dataframe into train/test/val x/y arrays. Based on folds predefined in a dataset
    """
    train = df[df['fold'] != fold]
    val_fold = train.fold.sample(n=1).values[0]
    val = train[train['fold'] == val_fold]
    train = train[train['fold'] != val_fold]
    test = df[df['fold'] == fold]
    train_x = np.stack([x for x in train['r'].values])
    train_y = train[classes]
    test = df[df['fold'] == fold]
    test_x = np.stack([x for x in test['r'].values])
    test_y = test[classes]
    val_x = np.stack([x for x in val['r'].values])
    val_y = val[classes]
    # print(len(val))
    return train_x, train_y, test_x, test_y, val_x, val_y

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import time
import os

def file_control(fpath):
    """
    This function is used to clear the file storing weights of the model. If not done,
    ModelCheckpoint will capture best performance from the file, ignoring consequent experiment runs.
    """
    if os.path.isfile(fpath):
        open(fpath, 'w').close()
        os.remove(fpath)
        print('cleared old weights')
    else:
        print('already clear')


def build_model(nc=1, shape=(224, 224, 3), base_trainable=False, backbone = "EfNetV2B1", weights = None):
    """
    This is a function creating a {backbone}-based model. The base model itself is pulled from tfhub/tf repo, Following paraneters are to be specified when calling this function:
    nc -- number of classes, or "width" of the model head
    shape -- image size provided for an input
    base_trainable -- layers of the base model (EfficientNetV2B2 in this case) can be locked as trainable or not, affecting model performance and training time.
    backbone - type of backbone to use
    weights - None or 'imagenet'
    """
    if backbone == "EfNetV2B1":
      base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B1(include_top=False, input_shape=shape, pooling='max', weights=weights)
    elif backbone == "EfNetV2B2":
      base_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    elif backbone == "ResNet152":
      base_model=tf.keras.applications.resnet_v2.ResNet152V2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    else:
      base_model=tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=shape, pooling='max', weights=weights)
    for layer in base_model.layers:
        layer.trainable = base_trainable
    dropout_layer = tf.keras.layers.Dropout(
        0.5, name="Dropout1")(base_model.output)
    clf_layer = tf.keras.layers.Dense(
        256, activation="sigmoid", name='Dense1')(dropout_layer)
    dropout_layer2 = tf.keras.layers.Dropout(0.4, name="Dropout2")(clf_layer)
    clf_layer2 = tf.keras.layers.Dense(
        128, activation="sigmoid", name='Dense2')(dropout_layer2)
    dropout_layer3 = tf.keras.layers.Dropout(0.1, name="Dropout3")(clf_layer2)
    clf_layer3 = tf.keras.layers.Dense(
        nc, activation="sigmoid", name='Dense3')(dropout_layer3)
    model = tf.keras.Model(base_model.input, clf_layer3)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return model


def create_dataset(x, y, batch_size=64, shuffle=True):
  """ This function is converting pandas dataframe into tensorflow dataset.
  This step should allow dynamic dataset loading into GPU RAM.
  """
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(batch_size)
    return dataset


def run_model(train_x, train_y, test_x, test_y, val_x, val_y, backbone = "ResNet50", batch_size = 64):
    """ A Wrap-up function to run the model. It will be called from the main function in the next cell
    Basically, it is creating a model, training it, and then returning the predictions on the test set.
    """
    file_control(path + 'm.h5')
    image_shape = np.shape(train_x[0])
    num_classes = np.shape(train_y)[1]
    # All models are given a chance to train for up to 100 epochs. However, in order to prevent overfit,
    # Models will be stopped if their validation metric is not imporving for at least 1% for over 8 epochs
    sb = tf.keras.callbacks.ModelCheckpoint(
        path+'m.h5', monitor='val_loss', verbose=1, save_best_only=True)
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0.01, patience=8, restore_best_weights=False)
    train_dataset = create_dataset(train_x, train_y, batch_size = batch_size)
    val_dataset = create_dataset(val_x, val_y, shuffle=False, batch_size = batch_size)
    model = build_model(nc=num_classes, base_trainable=True, shape=image_shape, backbone = backbone)
    model.fit(train_x, train_y, batch_size=64, epochs=100, verbose=0,
              callbacks=[es, sb], validation_data=(val_x, val_y))
    model.load_weights(path + 'm.h5')
    test_dataset = create_dataset(test_x, None, shuffle=False,batch_size = batch_size)
    predictions = model.predict(test_dataset)
    return predictions

In [ ]:
def run_experiments(name, backbone = "RsNet50", batch_size = 64):
    """
    In this function, we are running the model for each fold of the dataset. The results are stored in the results.pkl file.
    name is specyfing the version of the dataset to run experiments on.
    """
    start = time.time()
    print(f'running experiment for {name}')
    results = pd.read_pickle(path + 'results.pkl')
    df = pd.concat(build_df(name), ignore_index=True, axis=0)
    r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
    for fold in results.fold.unique():
        train_x, train_y, test_x, test_y, val_x, val_y = split(df, fold)
        test_link = df[df['fold'] == fold][[
            'slice_file_name', 'fold', 'source']]
        pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y, backbone = backbone)
        test_link[name] = pd.DataFrame(
            pred, columns=classes).idxmax(axis=1).to_numpy()
        r = pd.concat(
            [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
    results = pd.merge(results, r,  how='left', left_on=[
                       'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
    results.to_pickle(path + "results.pkl")
    end = time.time()
    print(f'done in {str(round(end - start,2))}')

In [ ]:
run_experiments(names[0], backbone = backbone)

running experiment for dist
cleared old weights

Epoch 1: val_loss improved from inf to 2.37302, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.37302

Epoch 3: val_loss did not improve from 2.37302

Epoch 4: val_loss did not improve from 2.37302

Epoch 5: val_loss did not improve from 2.37302

Epoch 6: val_loss did not improve from 2.37302

Epoch 7: val_loss did not improve from 2.37302

Epoch 8: val_loss did not improve from 2.37302

Epoch 9: val_loss did not improve from 2.37302
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.72819, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.72819

Epoch 3: val_loss did not improve from 2.72819

Epoch 4: val_loss did not improve from 2.72819

Epoch 5: val_loss did not improve from 2.72819

Epoch 6: val_loss improved from 2.72819 to 1.91724, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.91724

Epoch 8: val_loss did not improve from 1.91724

Epoch 9: val_loss did not improve from 1.91724

Epoch 10: val_loss did not improve from 1.91724

Epoch 11: val_loss did not improve from 1.91724

Epoch 12: val_loss did not improve from 1.91724

Epoch 13: val_loss improved from 1.91724 to 1.83417, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss did not improve from 1.83417

Epoch 15: val_loss did not improve from 1.83417

Epoch 16: val_loss did not improve from 1.83417

Epoch 17: val_loss did not improve from 1.83417

Epoch 18: val_loss did not improve from 1.83417

Epoch 19: val_loss did not impro

...


Epoch 2: val_loss did not improve from 2.41803

Epoch 3: val_loss did not improve from 2.41803

Epoch 4: val_loss did not improve from 2.41803

Epoch 5: val_loss did not improve from 2.41803

Epoch 6: val_loss did not improve from 2.41803

Epoch 7: val_loss did not improve from 2.41803

Epoch 8: val_loss did not improve from 2.41803

Epoch 9: val_loss did not improve from 2.41803
6/6 [==============================] - 2s 145ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.43848, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.43848

Epoch 3: val_loss did not improve from 2.43848

Epoch 4: val_loss did not improve from 2.43848

Epoch 5: val_loss did not improve from 2.43848

Epoch 6: val_loss did not improve from 2.43848

Epoch 7: val_loss did not improve from 2.43848

Epoch 8: val_loss did not improve from 2.43848

Epoch 9: val_loss did not improve from 2.43848
6/6 [==============================] - 1s 57ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.51621, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.51621

Epoch 3: val_loss did not improve from 2.51621

Epoch 4: val_loss did not improve from 2.51621

Epoch 5: val_loss did not improve from 2.51621

Epoch 6: val_loss did not improve from 2.51621

Epoch 7: val_loss did not improve from 2.51621

Epoch 8: val_loss did not improve from 2.51621

Epoch 9: val_loss did not improve from 2.51621
6/6 [==============================] - 39s 56ms/step
done in 1759.19


In [ ]:
run_experiments(names[1],backbone = backbone)

running experiment for mixup
cleared old weights

Epoch 1: val_loss improved from inf to 2.48083, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.48083

Epoch 3: val_loss did not improve from 2.48083

Epoch 4: val_loss did not improve from 2.48083

Epoch 5: val_loss did not improve from 2.48083

Epoch 6: val_loss did not improve from 2.48083

Epoch 7: val_loss did not improve from 2.48083

Epoch 8: val_loss did not improve from 2.48083

Epoch 9: val_loss improved from 2.48083 to 1.99550, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.99550

Epoch 11: val_loss did not improve from 1.99550

Epoch 12: val_loss did not improve from 1.99550

Epoch 13: val_loss did not improve from 1.99550

Epoch 14: val_loss did not improve from 1.99550

Epoch 15: val_loss did not improve from 1.99550

Epoch 16: val_loss did not improve from 1.99550

Epoch 17: val_loss did not improve from 1.99550
6/6 [==============================] - 1s 57ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.64853, saving model to /content/gdrive/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.64853

Epoch 3: val_loss did not improve from 2.64853

Epoch 4: val_loss did not improve from 2.64853

Epoch 5: val_loss improved from 2.64853 to 2.47025, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 2.47025

Epoch 7: val_loss did not improve from 2.47025

Epoch 8: val_loss did not improve from 2.47025

Epoch 9: val_loss did not improve from 2.47025

Epoch 10: val_loss did not improve from 2.47025

Epoch 11: val_loss did not improve from 2.47025

Epoch 12: val_loss improved from 2.47025 to 2.43407, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss improved from 2.43407 to 2.17018, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss did not improve from 2.17018

Epoch 15: val_loss did not improve from 2.17018

Epoch 16: val_loss did not improve from 2.17018

Epoch 17: val_loss did not improve from 2.17018

Epoch 18: 

...


Epoch 2: val_loss did not improve from 2.43921

Epoch 3: val_loss did not improve from 2.43921

Epoch 4: val_loss did not improve from 2.43921

Epoch 5: val_loss did not improve from 2.43921

Epoch 6: val_loss did not improve from 2.43921

Epoch 7: val_loss did not improve from 2.43921

Epoch 8: val_loss did not improve from 2.43921

Epoch 9: val_loss did not improve from 2.43921
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.44203, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.44203

Epoch 3: val_loss did not improve from 2.44203

Epoch 4: val_loss did not improve from 2.44203

Epoch 5: val_loss improved from 2.44203 to 2.43111, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 2.43111

Epoch 7: val_loss did not improve from 2.43111

Epoch 8: val_loss did not improve from 2.43111

Epoch 9: val_loss did not improve from 2.43111

Epoch 10: val_loss did not improve from 2.43111

Epoch 11: val_loss did not improve from 2.43111

Epoch 12: val_loss did not improve from 2.43111

Epoch 13: val_loss did not improve from 2.43111
6/6 [==============================] - 1s 57ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.45218, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.45218

Epoch 3: val_loss did not improve from 2.45218

Epoch 4: val_loss did not improve from 2.45218

Epoch 5: val_loss improved from 2.45218 to 2.40241, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 2.40241 to 2.29496, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 2.29496

Epoch 8: val_loss did not improve from 2.29496

Epoch 9: val_loss did not improve from 2.29496

Epoch 10: val_loss did not improve from 2.29496

Epoch 11: val_loss did not improve from 2.29496

Epoch 12: val_loss did not improve from 2.29496

Epoch 13: val_loss did not improve from 2.29496

Epoch 14: val_loss improved from 2.29496 to 2.18920, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 15: val_loss did not improve from 2.18920

Epoch 16: val_loss did not improve from 2.18920

Epoch 17: val_loss did not improve from 2.18920

Epoch 18: 

In [ ]:
run_experiments(names[2],backbone = backbone)

running experiment for imixup
cleared old weights

Epoch 1: val_loss improved from inf to 2.38907, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.38907

Epoch 3: val_loss did not improve from 2.38907

Epoch 4: val_loss improved from 2.38907 to 2.35374, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.35374

Epoch 6: val_loss did not improve from 2.35374

Epoch 7: val_loss improved from 2.35374 to 2.27097, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 2.27097

Epoch 9: val_loss improved from 2.27097 to 2.20245, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 2.20245

Epoch 11: val_loss did not improve from 2.20245

Epoch 12: val_loss did not improve from 2.20245

Epoch 13: val_loss did not improve from 2.20245

Epoch 14: val_loss did not improve from 2.20245

Epoch 15: val_loss did not improve from 2.20245

Epoch 16: val_loss did not improve from 2.20245

Epoch 17: val_loss improved from 2.20245 to 1.65332, saving

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.37013

Epoch 3: val_loss did not improve from 2.37013

Epoch 4: val_loss improved from 2.37013 to 2.33932, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.33932

Epoch 6: val_loss did not improve from 2.33932

Epoch 7: val_loss improved from 2.33932 to 1.94374, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.94374

Epoch 9: val_loss did not improve from 1.94374

Epoch 10: val_loss did not improve from 1.94374

Epoch 11: val_loss did not improve from 1.94374

Epoch 12: val_loss did not improve from 1.94374

Epoch 13: val_loss did not improve from 1.94374

Epoch 14: val_loss did not improve from 1.94374

Epoch 15: val_loss did not improve from 1.94374
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.36535, saving model to /content/gdrive/MyDrive/Colab Notebooks/dat

...


Epoch 2: val_loss did not improve from 2.38191

Epoch 3: val_loss did not improve from 2.38191

Epoch 4: val_loss improved from 2.38191 to 2.32801, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.32801

Epoch 6: val_loss did not improve from 2.32801

Epoch 7: val_loss did not improve from 2.32801

Epoch 8: val_loss improved from 2.32801 to 2.25685, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 2.25685

Epoch 10: val_loss did not improve from 2.25685

Epoch 11: val_loss did not improve from 2.25685

Epoch 12: val_loss improved from 2.25685 to 2.19391, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss did not improve from 2.19391

Epoch 14: val_loss did not improve from 2.19391

Epoch 15: val_loss did not improve from 2.19391

Epoch 16: val_loss improved from 2.19391 to 1.95272, saving model to /content/gdrive/MyDrive/Colab Notebooks

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.38676

Epoch 3: val_loss did not improve from 2.38676

Epoch 4: val_loss did not improve from 2.38676

Epoch 5: val_loss improved from 2.38676 to 2.27408, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 2.27408

Epoch 7: val_loss did not improve from 2.27408

Epoch 8: val_loss did not improve from 2.27408

Epoch 9: val_loss improved from 2.27408 to 1.97535, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.97535

Epoch 11: val_loss did not improve from 1.97535

Epoch 12: val_loss did not improve from 1.97535

Epoch 13: val_loss improved from 1.97535 to 1.76363, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss did not improve from 1.76363

Epoch 15: val_loss did not improve from 1.76363

Epoch 16: val_loss did not improve from 1.76363

Epoch 17: val_loss did not improve from 1.76363

Epoch 18: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.43148

Epoch 3: val_loss did not improve from 2.43148

Epoch 4: val_loss did not improve from 2.43148

Epoch 5: val_loss did not improve from 2.43148

Epoch 6: val_loss improved from 2.43148 to 2.15908, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 2.15908

Epoch 8: val_loss did not improve from 2.15908

Epoch 9: val_loss did not improve from 2.15908

Epoch 10: val_loss did not improve from 2.15908

Epoch 11: val_loss did not improve from 2.15908

Epoch 12: val_loss did not improve from 2.15908

Epoch 13: val_loss did not improve from 2.15908

Epoch 14: val_loss did not improve from 2.15908
6/6 [==============================] - 1s 56ms/step
done in 2301.25


In [ ]:
run_experiments(names[3],backbone = backbone)

running experiment for room
cleared old weights

Epoch 1: val_loss improved from inf to 2.45539, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.45539

Epoch 3: val_loss did not improve from 2.45539

Epoch 4: val_loss did not improve from 2.45539

Epoch 5: val_loss did not improve from 2.45539

Epoch 6: val_loss did not improve from 2.45539

Epoch 7: val_loss improved from 2.45539 to 2.42564, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 2.42564

Epoch 9: val_loss did not improve from 2.42564

Epoch 10: val_loss did not improve from 2.42564

Epoch 11: val_loss did not improve from 2.42564

Epoch 12: val_loss did not improve from 2.42564

Epoch 13: val_loss improved from 2.42564 to 2.25795, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss did not improve from 2.25795

Epoch 15: val_loss did not improve from 2.25795

Epoch 16: val_loss did not improve from 2.25795

Epoch 17: val_loss did not improve from 2.25795

Epoch 18: val_loss did not improve from 2.25795

Epoch 19: val_loss improved from

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.36070

Epoch 3: val_loss did not improve from 2.36070

Epoch 4: val_loss did not improve from 2.36070

Epoch 5: val_loss did not improve from 2.36070

Epoch 6: val_loss did not improve from 2.36070

Epoch 7: val_loss did not improve from 2.36070

Epoch 8: val_loss did not improve from 2.36070

Epoch 9: val_loss did not improve from 2.36070
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.58896, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...


Epoch 2: val_loss did not improve from 2.55090

Epoch 3: val_loss did not improve from 2.55090

Epoch 4: val_loss did not improve from 2.55090

Epoch 5: val_loss did not improve from 2.55090

Epoch 6: val_loss did not improve from 2.55090

Epoch 7: val_loss improved from 2.55090 to 2.49183, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 2.49183 to 2.45484, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 2.45484

Epoch 10: val_loss did not improve from 2.45484

Epoch 11: val_loss did not improve from 2.45484

Epoch 12: val_loss improved from 2.45484 to 2.14505, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss improved from 2.14505 to 2.05163, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 14: val_loss improved from 2.05163 to 1.63781, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 15: val_l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.39393

Epoch 3: val_loss did not improve from 2.39393

Epoch 4: val_loss did not improve from 2.39393

Epoch 5: val_loss did not improve from 2.39393

Epoch 6: val_loss did not improve from 2.39393

Epoch 7: val_loss did not improve from 2.39393

Epoch 8: val_loss did not improve from 2.39393

Epoch 9: val_loss did not improve from 2.39393
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.59867, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.59867

Epoch 3: val_loss did not improve from 2.59867

Epoch 4: val_loss did not improve from 2.59867

Epoch 5: val_loss did not improve from 2.59867

Epoch 6: val_loss did not improve from 2.59867

Epoch 7: val_loss did not improve from 2.59867

Epoch 8: val_loss improved from 2.59867 to 2.23944, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 2.23944

Epoch 10: val_loss improved from 2.23944 to 2.11637, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 2.11637

Epoch 12: val_loss improved from 2.11637 to 1.63988, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss did not improve from 1.63988

Epoch 14: val_loss did not improve from 1.63988

Epoch 15: val_loss did not improve from 1.63988

Epoch 16: val_loss did not improve from 1.63988

Epoch 17: val_loss did not improve from 1.63988

Epoch 18: 

In [ ]:
run_experiments(names[4],backbone = backbone)

running experiment for warp
already clear

Epoch 1: val_loss improved from inf to 2.47354, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.47354

Epoch 3: val_loss did not improve from 2.47354

Epoch 4: val_loss did not improve from 2.47354

Epoch 5: val_loss did not improve from 2.47354

Epoch 6: val_loss did not improve from 2.47354

Epoch 7: val_loss improved from 2.47354 to 2.18019, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 2.18019

Epoch 9: val_loss did not improve from 2.18019

Epoch 10: val_loss improved from 2.18019 to 2.14469, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 2.14469

Epoch 12: val_loss did not improve from 2.14469

Epoch 13: val_loss did not improve from 2.14469

Epoch 14: val_loss did not improve from 2.14469

Epoch 15: val_loss improved from 2.14469 to 2.08115, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 16: val_loss did not improve from 2.08115

Epoch 17: val_loss improved from 2.08115 to 1.87824, saving

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.42621

Epoch 3: val_loss did not improve from 2.42621

Epoch 4: val_loss did not improve from 2.42621

Epoch 5: val_loss did not improve from 2.42621

Epoch 6: val_loss did not improve from 2.42621

Epoch 7: val_loss did not improve from 2.42621

Epoch 8: val_loss did not improve from 2.42621

Epoch 9: val_loss did not improve from 2.42621
6/6 [==============================] - 1s 55ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.31452, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...


Epoch 2: val_loss did not improve from 2.50355

Epoch 3: val_loss improved from 2.50355 to 2.24475, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 2.24475

Epoch 5: val_loss did not improve from 2.24475

Epoch 6: val_loss did not improve from 2.24475

Epoch 7: val_loss did not improve from 2.24475

Epoch 8: val_loss did not improve from 2.24475

Epoch 9: val_loss did not improve from 2.24475

Epoch 10: val_loss did not improve from 2.24475

Epoch 11: val_loss did not improve from 2.24475
6/6 [==============================] - 1s 55ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.37701, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.37701

Epoch 3: val_loss did not improve from 2.37701

Epoch 4: val_loss did not improve from 2.37701

Epoch 5: val_loss did not improve from 2.37701

Epoch 6: val_loss did not improve from 2.37701

Epoch 7: val_loss improved from 2.37701 to 2.35131, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 2.35131 to 2.27088, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 2.27088

Epoch 10: val_loss improved from 2.27088 to 2.09289, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 2.09289

Epoch 12: val_loss did not improve from 2.09289

Epoch 13: val_loss did not improve from 2.09289

Epoch 14: val_loss improved from 2.09289 to 1.73979, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 15: val_loss did not improve from 1.73979

Epoch 16: val_loss did not improve fr

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.50006

Epoch 3: val_loss did not improve from 2.50006

Epoch 4: val_loss did not improve from 2.50006

Epoch 5: val_loss did not improve from 2.50006

Epoch 6: val_loss did not improve from 2.50006

Epoch 7: val_loss did not improve from 2.50006

Epoch 8: val_loss did not improve from 2.50006

Epoch 9: val_loss did not improve from 2.50006
6/6 [==============================] - 1s 57ms/step
done in 1848.83


In [ ]:
run_experiments(names[5], backbone = backbone)

running experiment for delay
cleared old weights

Epoch 1: val_loss improved from inf to 2.61096, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.61096

Epoch 3: val_loss did not improve from 2.61096

Epoch 4: val_loss improved from 2.61096 to 2.42746, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 2.42746 to 2.30224, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 2.30224

Epoch 7: val_loss did not improve from 2.30224

Epoch 8: val_loss did not improve from 2.30224

Epoch 9: val_loss did not improve from 2.30224

Epoch 10: val_loss did not improve from 2.30224

Epoch 11: val_loss did not improve from 2.30224

Epoch 12: val_loss did not improve from 2.30224

Epoch 13: val_loss did not improve from 2.30224
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.47672, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.47672

Epoch 3: val_loss did not improve from 2.47672

Epoch 4: val_loss did not improve from 2.47672

Epoch 5: val_loss improved from 2.47672 to 2.39681, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss did not improve from 2.39681

Epoch 7: val_loss did not improve from 2.39681

Epoch 8: val_loss did not improve from 2.39681

Epoch 9: val_loss improved from 2.39681 to 2.20446, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 2.20446 to 1.90520, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.90520

Epoch 12: val_loss did not improve from 1.90520

Epoch 13: val_loss did not improve from 1.90520

Epoch 14: val_loss improved from 1.90520 to 1.88809, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 15: val_loss improved from 1.88809 to 1.77358, saving model to /content/gdrive/M

...


Epoch 2: val_loss did not improve from 2.42502

Epoch 3: val_loss did not improve from 2.42502

Epoch 4: val_loss did not improve from 2.42502

Epoch 5: val_loss did not improve from 2.42502

Epoch 6: val_loss did not improve from 2.42502

Epoch 7: val_loss did not improve from 2.42502

Epoch 8: val_loss did not improve from 2.42502

Epoch 9: val_loss did not improve from 2.42502
6/6 [==============================] - 1s 56ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.58416, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.58416

Epoch 3: val_loss did not improve from 2.58416

Epoch 4: val_loss did not improve from 2.58416

Epoch 5: val_loss did not improve from 2.58416

Epoch 6: val_loss did not improve from 2.58416

Epoch 7: val_loss did not improve from 2.58416

Epoch 8: val_loss did not improve from 2.58416

Epoch 9: val_loss improved from 2.58416 to 2.29606, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 2.29606

Epoch 11: val_loss did not improve from 2.29606

Epoch 12: val_loss did not improve from 2.29606

Epoch 13: val_loss did not improve from 2.29606

Epoch 14: val_loss did not improve from 2.29606

Epoch 15: val_loss did not improve from 2.29606

Epoch 16: val_loss did not improve from 2.29606

Epoch 17: val_loss did not improve from 2.29606
6/6 [==============================] - 1s 55ms/step
cleared old weights

Epoch 1: val_loss improved from inf to 2.50416, saving model to /content/gdrive/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.50416

Epoch 3: val_loss did not improve from 2.50416

Epoch 4: val_loss did not improve from 2.50416

Epoch 5: val_loss did not improve from 2.50416

Epoch 6: val_loss improved from 2.50416 to 2.15214, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 2.15214

Epoch 8: val_loss did not improve from 2.15214

Epoch 9: val_loss improved from 2.15214 to 1.87504, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.87504

Epoch 11: val_loss did not improve from 1.87504

Epoch 12: val_loss did not improve from 1.87504

Epoch 13: val_loss did not improve from 1.87504

Epoch 14: val_loss did not improve from 1.87504

Epoch 15: val_loss did not improve from 1.87504

Epoch 16: val_loss did not improve from 1.87504

Epoch 17: val_loss did not improve from 1.87504
6/6 [==============================] - 1s 56ms/step
done in 1812.43


In [ ]:
run_experiments(names[6], backbone = backbone)

In [19]:
pd.read_pickle(path + 'results.pkl').head(5)

,slice_file_name,fold,class,source,dist,mixup,imixup,room,warp,delay,spectrum,all,image_only,audio_only,audio_only_nd
0,57320-0-0-39.wav,1,air_conditioner,o,drilling,street_music,engine_idling,dog_bark,air_conditioner,engine_idling,jackhammer,engine_idling,air_conditioner,siren,street_music
1,134717-0-0-6.wav,1,air_conditioner,o,drilling,street_music,engine_idling,air_conditioner,jackhammer,air_conditioner,jackhammer,engine_idling,siren,street_music,street_music
2,57320-0-0-31.wav,1,air_conditioner,o,drilling,street_music,children_playing,dog_bark,air_conditioner,engine_idling,jackhammer,air_conditioner,dog_bark,siren,street_music
3,134717-0-0-15.wav,1,air_conditioner,o,drilling,street_music,engine_idling,air_conditioner,air_conditioner,engine_idling,jackhammer,air_conditioner,siren,street_music,street_music
4,177621-0-0-27.wav,1,air_conditioner,o,drilling,street_music,engine_idling,air_conditioner,air_conditioner,engine_idling,jackhammer,engine_idling,engine_idling,street_music,street_music


In [ ]:
#Now it is time to see how various combinations of augmentation techniques are performing.
#The following cell is creating a function similar to run_experiments but for group of augmentation techniques

def run_grouped_experiments(group, group_name, backbone = backbone, batch_size = 64):
  """ This function is running experiments for a group of augmentation techniques, very similar to run_experiments function.
  group is a list of augmentation techniques to be used.
  group_name is a name of the group, which will be used to name the column in the results dataframe.
  backbone is a type of backbone to be used to build a model.
  batch_size is a batch size to be used for training.
  """
  start = time.time()
  results = pd.read_pickle(path + 'results.pkl')
  dfs = []
  for n in group:
      if n != 'spectrum':
          dfs.append(build_df(n)[1])
      else:
          dfs.append(build_df(n)[0])
  df = pd.concat(dfs, ignore_index=True, axis=0)
  r = pd.DataFrame(columns=['slice_file_name', 'fold', 'source', name])
  for fold in results.fold.unique():
      train = df[df['fold'] != fold]
      val_fold = train.fold.sample(n=1).values[0]
      val = train[train['fold'] == val_fold]
      val = val[val.source == 'o']
      train = train[train['fold'] != val_fold]
      test = df[df['fold'] == fold]
      test = test[test.source == 'o']
      train_x = np.stack([x for x in train['r'].values])
      train_y = train[classes]
      test = df[df['fold'] == fold]
      test_x = np.stack([x for x in test['r'].values])
      test_y = test[classes]
      val_x = np.stack([x for x in val['r'].values])
      val_y = val[classes]
      test_link = test[['slice_file_name', 'fold', 'source']]
      pred = run_model(train_x, train_y, test_x, test_y, val_x, val_y, backbone = backbone)
      test_link[name] = pd.DataFrame(
          pred, columns=classes).idxmax(axis=1).to_numpy()
      r = pd.concat(
          [r, test_link[['slice_file_name', 'fold', 'source', name]]], ignore_index=True)
  results = pd.merge(results, r,  how='left', left_on=[
                    'slice_file_name', 'fold', 'source'], right_on=['slice_file_name', 'fold', 'source'])
  results.to_pickle(path + "results.pkl")
  end = time.time()
  print(f'done in {str(round(end - start,2))}')

In [ ]:
run_grouped_experiments(names,"all",backbone = backbone, batch_size = 64)

cleared old weights

Epoch 1: val_loss improved from inf to 2.60920, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.60920

Epoch 3: val_loss improved from 2.60920 to 2.27392, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 2.27392 to 2.26172, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.26172

Epoch 6: val_loss improved from 2.26172 to 1.88169, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss improved from 1.88169 to 1.70723, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss improved from 1.70723 to 1.42418, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 9: val_loss did not improve from 1.42418

Epoch 10: val_loss did not improve from 1.42418

Epoch 11: val_loss improved from 1.42418 to 0.98346, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss did not improve from 0.98346

Epoch 13: val_loss did not improve from 0.

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.53897, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.95069, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.95069 to 2.77147, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss improved from 2.77147 to 2.36732, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 2.36732 to 2.28434, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.28434

Epoch 6: val_loss did not improve from 2.28434

Epoch 7: val_loss improved from 2.28434 to 1.51696, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.51696

Epoch 9: val_loss did not improve from 1.51696

Epoch 10: val_loss did not improve from 1.51696

Epoch 11: val_loss improved from 1.51696 to 1.38651, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 12: val_loss did not improve from 1.38651

Epoch 13: val_loss did not improve from 1.38651

Epoch 14: val_loss did not improve from 1.38651

Epoch 15: val_l

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


In [ ]:
run_grouped_experiments(['imixup','warp','spectrum'],"image_only",backbone = backbone, batch_size = 64)

cleared old weights

Epoch 1: val_loss improved from inf to 2.51082, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.51082

Epoch 3: val_loss improved from 2.51082 to 2.28577, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss improved from 2.28577 to 2.02350, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.02350

Epoch 6: val_loss did not improve from 2.02350

Epoch 7: val_loss did not improve from 2.02350

Epoch 8: val_loss did not improve from 2.02350

Epoch 9: val_loss did not improve from 2.02350

Epoch 10: val_loss improved from 2.02350 to 1.70840, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.70840

Epoch 12: val_loss did not improve from 1.70840

Epoch 13: val_loss did not improve from 1.70840

Epoch 14: val_loss did not improve from 1.70840

Epoch 15: val_loss did not improve from 1.70840

Epoch 16: val_loss did not improve from 1.70840

Epoch 17: val_loss did not improve from 1.70840

Epoch 18: 

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.68871, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.56303, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.56303

Epoch 3: val_loss did not improve from 2.56303

Epoch 4: val_loss improved from 2.56303 to 2.44728, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss improved from 2.44728 to 2.36759, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 2.36759 to 1.76816, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 1.76816

Epoch 8: val_loss did not improve from 1.76816

Epoch 9: val_loss did not improve from 1.76816

Epoch 10: val_loss did not improve from 1.76816

Epoch 11: val_loss did not improve from 1.76816

Epoch 12: val_loss improved from 1.76816 to 1.12982, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 13: val_loss did not improve from 1.12982

Epoch 14: val_loss did not improve from 1.12982

Epoch 15: val_loss did not improve from 1.12982

Epoch 16: val_loss did not improve fr

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


In [13]:
run_grouped_experiments(['dist', 'mixup', 'room','delay', 'spectrum'],"audio_only",backbone = backbone, batch_size = 64)


cleared old weights

Epoch 1: val_loss improved from inf to 2.72122, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.72122 to 2.46126, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 3: val_loss did not improve from 2.46126

Epoch 4: val_loss did not improve from 2.46126

Epoch 5: val_loss did not improve from 2.46126

Epoch 6: val_loss did not improve from 2.46126

Epoch 7: val_loss improved from 2.46126 to 1.32940, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 8: val_loss did not improve from 1.32940

Epoch 9: val_loss did not improve from 1.32940

Epoch 10: val_loss did not improve from 1.32940

Epoch 11: val_loss did not improve from 1.32940

Epoch 12: val_loss did not improve from 1.32940

Epoch 13: val_loss did not improve from 1.32940

Epoch 14: val_loss did not improve from 1.32940

Epoch 15: val_loss did not improve from 1.32940
15/15 [==============================] - 2s 43ms/step


<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.63730, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.68334, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.68334

Epoch 3: val_loss did not improve from 2.68334

Epoch 4: val_loss did not improve from 2.68334

Epoch 5: val_loss improved from 2.68334 to 2.53535, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 6: val_loss improved from 2.53535 to 2.25556, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 7: val_loss did not improve from 2.25556

Epoch 8: val_loss did not improve from 2.25556

Epoch 9: val_loss improved from 2.25556 to 1.98942, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss improved from 1.98942 to 1.97670, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 11: val_loss did not improve from 1.97670

Epoch 12: val_loss did not improve from 1.97670

Epoch 13: val_loss did not improve from 1.97670

Epoch 14: val_loss did not improve from 1.97670

Epoch 15: val_loss did not improve from 1.97670

Epoch 16: val_loss did not improve fr

<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


In [14]:
run_grouped_experiments(['dist', 'mixup', 'room', 'spectrum'],"audio_only_nd",backbone = backbone, batch_size = 64)

cleared old weights

Epoch 1: val_loss improved from inf to 2.77243, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.77243

Epoch 3: val_loss improved from 2.77243 to 2.47702, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 4: val_loss did not improve from 2.47702

Epoch 5: val_loss did not improve from 2.47702

Epoch 6: val_loss did not improve from 2.47702

Epoch 7: val_loss did not improve from 2.47702

Epoch 8: val_loss did not improve from 2.47702

Epoch 9: val_loss improved from 2.47702 to 1.48798, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.48798

Epoch 11: val_loss did not improve from 1.48798

Epoch 12: val_loss did not improve from 1.48798

Epoch 13: val_loss did not improve from 1.48798

Epoch 14: val_loss did not improve from 1.48798

Epoch 15: val_loss did not improve from 1.48798

Epoch 16: val_loss did not improve from 1.48798

Epoch 17: val_loss did not improve from 1.48798
12/12 [==============================] - 2s 53ms/step


<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


cleared old weights

Epoch 1: val_loss improved from inf to 2.77043, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


...

cleared old weights

Epoch 1: val_loss improved from inf to 2.71213, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 2.71213

Epoch 3: val_loss did not improve from 2.71213

Epoch 4: val_loss improved from 2.71213 to 2.66102, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 5: val_loss did not improve from 2.66102

Epoch 6: val_loss did not improve from 2.66102

Epoch 7: val_loss did not improve from 2.66102

Epoch 8: val_loss did not improve from 2.66102

Epoch 9: val_loss improved from 2.66102 to 1.47490, saving model to /content/gdrive/MyDrive/Colab Notebooks/data/m.h5

Epoch 10: val_loss did not improve from 1.47490

Epoch 11: val_loss did not improve from 1.47490

Epoch 12: val_loss did not improve from 1.47490

Epoch 13: val_loss did not improve from 1.47490

Epoch 14: val_loss did not improve from 1.47490

Epoch 15: val_loss did not improve from 1.47490

Epoch 16: val_loss did not improve from 1.47490

Epoch 17: val_loss did not improve from 1.47490
12/12 [==============================] - 1s 35ms/step


<ipython-input-10-0ddd3b46834f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_link[name] = pd.DataFrame(


done in 2215.77
